In [100]:
import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn import datasets
from numpy import array
from pandas import DataFrame
from pandas import concat
from keras.models import model_from_json
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder


In [101]:
def rmse(objetivo, estimaciones):
    return np.sqrt(metrics.mean_squared_error(objetivo, estimaciones)
                  )

In [102]:
def  ajustested_r2(objetivo, estimaciones,n,k):
    r2=metrics.r2_score(objetico,estimaciones)
    return 1-(1-r2)*(n-1)/(n-k-1)

In [103]:
def evaluar_modelo(objetivo,estimaciones,n,k):
    return {
        "rmse":rmse(objetivo,estimaciones),
        "mae": metrics.mean_absolute_error(objetivo,estimaciones),
        "ajustested_r2": ajustested_r2(objetivo,estimaciones,n,k)
    }

In [104]:
#Cargo el modelo antes generado.
json_file = open('../Modelamiento/model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# Cargo el nuevo modelo
loaded_model.load_weights("../Modelamiento/LTSM_Cali_3meses.h5")
loaded_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 50)                10800     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 51        
Total params: 10,851
Trainable params: 10,851
Non-trainable params: 0
_________________________________________________________________


In [21]:
#Prueba manual con los datos 
#PM10
#DirVien
#H2S
#Humedad
#Lluvia
#NO2
#O3
#CO
#RSolar
#SO2
#Tasa_crecimiento_general
#Temp
#VelVien
x_input = np.array([20,148.6000061,0,72.69999695,1.269999981,0,17.64,2851.524414,
                    302.7999878,12.91359997,0.097400002,24.89999962,1.299999952])
print(x_input.shape)

df = pd.DataFrame({ 'Motos_vehiculos_particulares':    [348066.000,348066.000, 348066, 348066],
                    'Lluvia': [0.939613,0.382813,0.263171911239624,0.572834014892578],
                    'Poblacion': [2244668,2244668,2244668, 2244668]
                   })



(13,)


In [23]:
df.columns

Index(['Motos_vehiculos_particulares', 'Lluvia', 'Poblacion'], dtype='object')

In [24]:
reframed = series_to_supervised(df, 1, 1)
reframed

,var1(t-1),var2(t-1),var3(t-1),var1(t),var2(t),var3(t)
1,348066.0,0.939613,2244668.0,348066.0,0.382813,2244668
2,348066.0,0.382813,2244668.0,348066.0,0.263172,2244668
3,348066.0,0.263172,2244668.0,348066.0,0.572834,2244668


In [25]:

scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(df)
reframed = series_to_supervised(scaled, 1, 1)
print(reframed.head())



   var1(t-1)  var2(t-1)  var3(t-1)  var1(t)   var2(t)  var3(t)
1        0.0   1.000000        0.0      0.0  0.176868      0.0
2        0.0   0.176868        0.0      0.0  0.000000      0.0
3        0.0   0.000000        0.0      0.0  0.457781      0.0


In [13]:
values = df.values
values = values.reshape((values.shape[0], 1, values.shape[1]))
yhat = loaded_model.predict(values, verbose=0)
print(yhat)

[[1.0459689]
 [1.0459689]]


In [105]:
dataOriginal = pd.read_excel('../PreparacionDatos/FloraTotal_3mes_3V_mas_4.xlsx' , sheet_name='FloraTotal' ,header=0)

In [64]:
dataOriginal

,Motos_vehiculos_particulares,Lluvia,Poblacion
0,348066.000,0.939613,2244668.00
1,348066.000,0.382813,2244668.00
2,348066.000,0.263172,2244668.00
3,348066.000,0.572834,2244668.00
4,391327.000,0.190382,2269653.00
5,391327.000,0.413592,2269653.00
6,391327.000,0.408934,2269653.00
7,391327.000,0.595508,2269653.00
8,438769.000,0.060729,2294653.00
9,438769.000,0.426957,2294653.00


In [106]:
values = dataOriginal.values
values = values.astype('float32')
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)


In [107]:
scaled_X.shape

(36, 3)

In [108]:
#Remodelar la entrada para que sea 3D [muestras, pasos de tiempo, características]
train_X = scaled_X.reshape((scaled_X.shape[0], 1, scaled_X.shape[1]))
train_X.shape

(36, 1, 3)

In [109]:
inv_yhat[:,0]

NameError: name 'inv_yhat' is not defined

In [110]:
yhat = loaded_model.predict(train_X, verbose=0)



print(yhat)

[[0.32860452]
 [0.22645335]
 [0.20525326]
 [0.26071233]
 [0.20275612]
 [0.24235775]
 [0.241521  ]
 [0.27535427]
 [0.19058816]
 [0.25535253]
 [0.19873528]
 [0.254062  ]
 [0.22272052]
 [0.23885418]
 [0.21376653]
 [0.2742918 ]
 [0.23857638]
 [0.24188511]
 [0.25032553]
 [0.26667464]
 [0.24781144]
 [0.24513495]
 [0.23037054]
 [0.23286283]
 [0.268369  ]
 [0.26629335]
 [0.23696315]
 [0.27114797]
 [0.23431398]
 [0.2637131 ]
 [0.25779676]
 [0.27058488]
 [0.45340002]
 [0.46682614]
 [0.4365161 ]
 [0.4228624 ]]


In [111]:
inverted_X = scaler.inverse_transform([yhat])

ValueError: Found array with dim 3. Estimator expected <= 2.

In [82]:
values = dataOriginal.values
train_X = values.reshape((values.shape[0], 1, values.shape[1]))
train_X.shape


(36, 1, 3)

In [83]:
yhat = loaded_model.predict(train_X, verbose=0)



print(yhat)

[[1.0459689]
 [1.0459689]
 [1.0459689]
 [1.0459689]
 [1.0459689]
 [1.0459689]
 [1.0459689]
 [1.0459689]
 [1.0459689]
 [1.0459689]
 [1.0459689]
 [1.0459689]
 [1.6356308]
 [1.6356308]
 [1.6356308]
 [1.6356308]
 [1.7393099]
 [1.7393099]
 [1.7393099]
 [1.7393099]
 [1.7393099]
 [1.7393099]
 [1.7393099]
 [1.7393099]
 [1.7393099]
 [1.7393099]
 [1.7393099]
 [1.7393099]
 [1.7393099]
 [1.7393099]
 [1.7393099]
 [1.7393099]
 [3.508834 ]
 [3.508834 ]
 [3.508834 ]
 [3.2241385]]


In [ ]:
df = pd.DataFrame({ 'PM10':    [20,20],
                    'DirVien': [148.6000061,148.6000061],
                    'H2S': [0,0],
                    'Humedad': [72.69999695,72.69999695],
                    'Lluvia': [1.269999981,1.269999981],
                    'NO2': [0,0],
                    'O3': [17.64,17.64],
                    'CO': [2851.524414,2851.524414],
                    'RSolar': [302.7999878,302.7999878],
                    'SO2': [12.91359997,12.91359997],
                    'Tasa_crecimiento_general': [0.90,0.90],
                    'Temp': [24.89999962,24.89999962],
                    'VelVien': [1.299999952,1.299999952]
                    
                   })
